In [ ]:
#spark.stop()
%run ../connection.python.py

In [4]:
## Digitalizzazione_info_interne
## codice cluster info_interne


## devo trasformare tutto in array per ML
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [5]:
result = spark.read.format('parquet').load("/user/riccardo.motta/Aggiornamento_Digitalizzazione_20180301_20180831/result1.parquet").persist()

In [6]:
matrice_df = result

In [8]:
matrice_df.show(10, False)

+------------------------------------------------+-------+--------------+--------------+---------------+-------------------+-------------------+-------------------+------------+-------------+--------------+-------------+-------------------+--------------------+-------------------+-------------+-------------+--------------+----------------+---------------+-------------+---------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-------------------+-----+-----+-------------+-----+-------+-----+----+-------+------+---+-------------+------------------+---------------+---------------+---------------+----------------+------------------+-----------------+---------------+-----------------+--------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+----------------------+-------------+----------------+-----------------+------------------+--------------

In [11]:
ignore = ["skyid"]
assembler = VectorAssembler(
  inputCols=[x for x in matrice_df.columns if x not in ignore],
  outputCol="features")

In [12]:
parsedData =  assembler.transform(matrice_df)
parsedData.printSchema()

root
 |-- skyid: string (nullable = true)
 |-- devices: double (nullable = true)
 |-- mobile_devices: double (nullable = true)
 |-- tablet_devices: double (nullable = true)
 |-- desktop_devices: double (nullable = true)
 |-- mobile_dev_perc: double (nullable = true)
 |-- tablet_dev_perc: double (nullable = true)
 |-- desktop_dev_perc: double (nullable = true)
 |-- total_visits: double (nullable = true)
 |-- mobile_visits: double (nullable = true)
 |-- desktop_visits: double (nullable = true)
 |-- tablet_visits: double (nullable = true)
 |-- mobile_visit_perc: double (nullable = true)
 |-- desktop_visit_perc: double (nullable = true)
 |-- tablet_visit_perc: double (nullable = true)
 |-- sunday_visits: double (nullable = true)
 |-- monday_visits: double (nullable = true)
 |-- tuesday_visits: double (nullable = true)
 |-- wednesday_visits: double (nullable = true)
 |-- thursday_visits: double (nullable = true)
 |-- friday_visits: double (nullable = true)
 |-- saturday_visits: double (null

In [13]:
## prendo solo colonne

parsedData = parsedData["skyid", "features"]
parsedData = parsedData.select( col("features").alias("features"), col("skyid").alias("label"))

In [14]:
# Trains a k-means model.
kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(parsedData)

In [16]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(parsedData)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 323184025428.08344


In [18]:
# Shows the result
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
  print(center)

Cluster Centers: 
[  6.48720111e+00   2.63992515e+00   9.76412620e-01   2.69641937e+00
   3.53234778e-01   1.40011747e-01   4.67215678e-01   3.99326536e+01
   2.36357682e+01   1.20205635e+01   4.27529552e+00   3.51812783e-01
   4.03251828e-01   1.00741576e-01   2.57616389e+01   2.57616389e+01
   2.57616389e+01   2.57616389e+01   2.57616389e+01   2.57616389e+01
   2.57616389e+01   6.93566831e-01   6.93566831e-01   6.93566831e-01
   6.93566831e-01   6.93566831e-01   6.93566831e-01   6.93566831e-01
   4.72955500e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   3.26047129e+00   8.72474419e+00
   2.17737841e+01   4.50908971e+00   3.78262216e+00   3.73350772e+00
   3.71052808e+00   3.80790870e+00   3.89144496e+00   4.22874362e+00
   3.15967118e-01   2.94386555e-01   2.90935763e-01   2.91331830e-01
   2.94514670e-01   2.95899871e-01   3.06976152e-01   4.75805454e-02
   0.00000000e+0

[  4.63266091e+01   4.05632703e+01   2.30251252e+00   3.32289464e+00
   8.73882375e-01   4.74026749e-02   7.41144460e-02   1.62731394e+03
   1.57900032e+03   2.27305179e+01   2.55807951e+01   9.70038530e-01
   1.37088854e-02   1.62509804e-02   1.00508187e+03   1.00508187e+03
   1.00508187e+03   1.00508187e+03   1.00508187e+03   1.00508187e+03
   1.00508187e+03   6.43828442e-01   6.43828442e-01   6.43828442e-01
   6.43828442e-01   6.43828442e-01   6.43828442e-01   6.43828442e-01
   6.73124179e-01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.96643618e+00   2.67038307e+01
   1.73704097e+02   2.59323580e+01   2.55570033e+01   2.55424119e+01
   2.53303617e+01   2.62885117e+01   2.63736785e+01   2.56797722e+01
   1.49464254e-01   1.47179903e-01   1.47178640e-01   1.45936627e-01
   1.51426970e-01   1.51941988e-01   1.47931578e-01   3.38686020e-02
   0.00000000e+00   0.00000000e+00

In [19]:
model.summary.clusterSizes

[2643340, 1, 6683, 52729, 303812]